In [383]:
import pandas as pd
import numpy as np

In [384]:
import io
%cd "C:\Users\ykkou\Desktop\cointab assessment\Assignment details"

C:\Users\ykkou\Desktop\cointab assessment\Assignment details


In [385]:
x_orderreport = pd.read_excel('Company X - Order Report.xlsx')

In [386]:
x_skumaster = pd.read_excel('Company X - SKU Master.xlsx')

In [387]:
x_orderreport.head(10)

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233
5,2001827036,GIFTBOX202002,1,COD,659
6,2001827036,8904223818638,2,COD,570
7,2001827036,SACHETS001,1,COD,350
8,2001825261,8904223819024,4,Prepaid,3636
9,2001825261,8904223819291,4,Prepaid,3907


In [388]:
x_skumaster = x_skumaster.drop_duplicates(subset=['SKU'])

In [389]:
x_skumaster

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [390]:
order_skumaster = x_orderreport.merge(x_skumaster,on = 'SKU',how = 'left')

In [391]:
order_skumaster

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),Weight (g)
0,2001827036,8904223818706,1,COD,233,127
1,2001827036,8904223819093,1,COD,233,150
2,2001827036,8904223819109,1,COD,233,100
3,2001827036,8904223818430,1,COD,470,165
4,2001827036,8904223819277,1,COD,233,350
...,...,...,...,...,...,...
395,2001806229,8904223818942,1,COD,286,133
396,2001806229,8904223818850,1,COD,563,240
397,2001806226,8904223818850,2,COD,1125,240
398,2001806210,8904223816214,1,COD,402,120


In [392]:
pincode_zones = pd.read_excel('Company X - Pincode Zones.xlsx')

In [393]:
pincode_zones

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [394]:
pincode_zones['ExternOrderNo'] = pd.DataFrame(order_skumaster.ExternOrderNo.unique())

In [395]:
pincode_zones.drop('Warehouse Pincode',axis=1,inplace=True)

In [396]:
delivery_table = order_skumaster.merge(pincode_zones,on='ExternOrderNo',how = 'left')

In [397]:
delivery_table.head(10)

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),Weight (g),Customer Pincode,Zone
0,2001827036,8904223818706,1,COD,233,127,507101,d
1,2001827036,8904223819093,1,COD,233,150,507101,d
2,2001827036,8904223819109,1,COD,233,100,507101,d
3,2001827036,8904223818430,1,COD,470,165,507101,d
4,2001827036,8904223819277,1,COD,233,350,507101,d
5,2001827036,GIFTBOX202002,1,COD,659,500,507101,d
6,2001827036,8904223818638,2,COD,570,137,507101,d
7,2001827036,SACHETS001,1,COD,350,10,507101,d
8,2001825261,8904223819024,4,Prepaid,3636,112,486886,d
9,2001825261,8904223819291,4,Prepaid,3907,112,486886,d


In [398]:
table1 = pd.DataFrame((delivery_table['Weight (g)']*delivery_table['Order Qty']).groupby(delivery_table.ExternOrderNo).sum()/1000).reset_index()

In [399]:
table1

,ExternOrderNo,0
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245
...,...,...
119,2001821995,0.477
120,2001822466,1.376
121,2001823564,0.672
122,2001825261,1.557


In [400]:
table1.rename(columns={0: 'Total Weight as per X(kg)'},inplace=True)

In [401]:
table1['Delivery Zone as per X'] = pd.DataFrame(delivery_table.Zone.groupby(delivery_table.ExternOrderNo).first()).reset_index().Zone

In [402]:
table1

,ExternOrderNo,Total Weight as per X(kg),Delivery Zone as per X
0,2001806210,0.220,b
1,2001806226,0.480,e
2,2001806229,0.500,b
3,2001806232,1.302,b
4,2001806233,0.245,b
...,...,...,...
119,2001821995,0.477,d
120,2001822466,1.376,b
121,2001823564,0.672,d
122,2001825261,1.557,d


In [403]:
zone = table1['Delivery Zone as per X']
weight = table1['Total Weight as per X(kg)']

In [404]:
conditions = [
    zone == 'a',
    zone == 'b',
    zone == 'c',
    zone == 'd',
    zone == 'e'
]

choices = [
    (weight/0.25),
    (weight/0.5),
    (weight/0.75),
   (weight/1.25),
    (weight/1.5)
]

In [405]:
x = pd.DataFrame(np.select(conditions,choices,0)).apply(np.ceil)[0]

In [406]:
conditions = [
    zone == 'a',
    zone == 'b',
    zone == 'c',
    zone == 'd',
    zone == 'e'
]

choices = [
    (x*0.25),
    (x*0.5),
    (x*0.75),
   (x*1.25),
    (x*1.5)
]

In [407]:
table1['Weight Slab as per X (kg)'] = pd.DataFrame(np.select(conditions,choices,0))

In [408]:
table1

,ExternOrderNo,Total Weight as per X(kg),Delivery Zone as per X,Weight Slab as per X (kg)
0,2001806210,0.220,b,0.50
1,2001806226,0.480,e,1.50
2,2001806229,0.500,b,0.50
3,2001806232,1.302,b,1.50
4,2001806233,0.245,b,0.50
...,...,...,...,...
119,2001821995,0.477,d,1.25
120,2001822466,1.376,b,1.50
121,2001823564,0.672,d,1.25
122,2001825261,1.557,d,2.50


In [409]:
courier_invoice = pd.read_excel('Courier Company - Invoice.xlsx')

In [410]:
courier_invoice

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,101.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,95.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,47.0


In [411]:
table1=table1.merge(courier_invoice,left_on='ExternOrderNo', right_on='Order ID',how = 'left')

In [412]:
table1

,ExternOrderNo,Total Weight as per X(kg),Delivery Zone as per X,Weight Slab as per X (kg),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,0.220,b,0.50,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,0.480,e,1.50,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,0.500,b,0.50,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,1.302,b,1.50,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0
4,2001806233,0.245,b,0.50,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,d,1.25,1091121183730,2001821995,0.50,121003,342008,d,Forward charges,45.4
120,2001822466,1.376,b,1.50,1091121305541,2001822466,1.10,121003,342301,d,Forward charges,135.0
121,2001823564,0.672,d,1.25,1091121666133,2001823564,0.70,121003,492001,d,Forward and RTO charges,172.8
122,2001825261,1.557,d,2.50,1091121981575,2001825261,1.60,121003,517128,d,Forward and RTO charges,345.0


In [413]:
table1.drop(['Order ID','Warehouse Pincode'],axis=1,inplace=True)

In [414]:
table1

,ExternOrderNo,Total Weight as per X(kg),Delivery Zone as per X,Weight Slab as per X (kg),AWB Code,Charged Weight,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,0.220,b,0.50,1091117221940,2.92,140604,b,Forward charges,174.5
1,2001806226,0.480,e,1.50,1091117222065,0.68,723146,d,Forward charges,90.2
2,2001806229,0.500,b,0.50,1091117222080,0.71,421204,d,Forward charges,90.2
3,2001806232,1.302,b,1.50,1091117222124,1.30,507101,d,Forward charges,140.0
4,2001806233,0.245,b,0.50,1091117222135,0.78,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...
119,2001821995,0.477,d,1.25,1091121183730,0.50,342008,d,Forward charges,45.4
120,2001822466,1.376,b,1.50,1091121305541,1.10,342301,d,Forward charges,135.0
121,2001823564,0.672,d,1.25,1091121666133,0.70,492001,d,Forward and RTO charges,172.8
122,2001825261,1.557,d,2.50,1091121981575,1.60,517128,d,Forward and RTO charges,345.0


In [415]:
table1 = table1.loc[:,['ExternOrderNo','AWB Code','Total Weight as per X(kg)','Weight Slab as per X (kg)','Charged Weight','Delivery Zone as per X',
                      'Zone','Billing Amount (Rs.)','Type of Shipment']]

In [416]:
table1

,ExternOrderNo,AWB Code,Total Weight as per X(kg),Weight Slab as per X (kg),Charged Weight,Delivery Zone as per X,Zone,Billing Amount (Rs.),Type of Shipment
0,2001806210,1091117221940,0.220,0.50,2.92,b,b,174.5,Forward charges
1,2001806226,1091117222065,0.480,1.50,0.68,e,d,90.2,Forward charges
2,2001806229,1091117222080,0.500,0.50,0.71,b,d,90.2,Forward charges
3,2001806232,1091117222124,1.302,1.50,1.30,b,d,140.0,Forward charges
4,2001806233,1091117222135,0.245,0.50,0.78,b,b,61.3,Forward charges
...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1.25,0.50,d,d,45.4,Forward charges
120,2001822466,1091121305541,1.376,1.50,1.10,b,d,135.0,Forward charges
121,2001823564,1091121666133,0.672,1.25,0.70,d,d,172.8,Forward and RTO charges
122,2001825261,1091121981575,1.557,2.50,1.60,d,d,345.0,Forward and RTO charges


In [417]:
zone = table1['Zone']
weight = table1['Charged Weight']

In [418]:
conditions = [
    zone == 'a',
    zone == 'b',
    zone == 'c',
    zone == 'd',
    zone == 'e'
]

choices = [
    (weight/0.25),
    (weight/0.5),
    (weight/0.75),
   (weight/1.25),
    (weight/1.5)
]

In [419]:
x = pd.DataFrame(np.select(conditions,choices,0)).apply(np.ceil)[0]

In [420]:
conditions = [
    zone == 'a',
    zone == 'b',
    zone == 'c',
    zone == 'd',
    zone == 'e'
]

choices = [
    (x*0.25),
    (x*0.5),
    (x*0.75),
   (x*1.25),
    (x*1.5)
]

In [421]:
table1['Weight Slab charged by Courier Company (Kg)'] = pd.DataFrame(np.select(conditions,choices,0))

In [422]:
table1

,ExternOrderNo,AWB Code,Total Weight as per X(kg),Weight Slab as per X (kg),Charged Weight,Delivery Zone as per X,Zone,Billing Amount (Rs.),Type of Shipment,Weight Slab charged by Courier Company (Kg)
0,2001806210,1091117221940,0.220,0.50,2.92,b,b,174.5,Forward charges,3.00
1,2001806226,1091117222065,0.480,1.50,0.68,e,d,90.2,Forward charges,1.25
2,2001806229,1091117222080,0.500,0.50,0.71,b,d,90.2,Forward charges,1.25
3,2001806232,1091117222124,1.302,1.50,1.30,b,d,140.0,Forward charges,2.50
4,2001806233,1091117222135,0.245,0.50,0.78,b,b,61.3,Forward charges,1.00
...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1.25,0.50,d,d,45.4,Forward charges,1.25
120,2001822466,1091121305541,1.376,1.50,1.10,b,d,135.0,Forward charges,1.25
121,2001823564,1091121666133,0.672,1.25,0.70,d,d,172.8,Forward and RTO charges,1.25
122,2001825261,1091121981575,1.557,2.50,1.60,d,d,345.0,Forward and RTO charges,2.50


In [423]:
courier_rates = pd.read_excel('Courier Company - Rates.xlsx')

In [424]:
courier_rates['Zone'] = ['a','b','c','d','e']

In [425]:
courier_rates

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,a,0.25,29.5,23.6,13.6,23.6
1,b,0.50,33.0,28.3,20.5,28.3
2,c,0.75,40.1,38.9,31.9,38.9
3,d,1.25,45.4,44.8,41.3,44.8
4,e,1.50,56.6,55.5,50.7,55.5


In [426]:
for_cost = table1[['ExternOrderNo','Weight Slab as per X (kg)','Delivery Zone as per X','Type of Shipment']]

In [427]:
for_cost

,ExternOrderNo,Weight Slab as per X (kg),Delivery Zone as per X,Type of Shipment
0,2001806210,0.50,b,Forward charges
1,2001806226,1.50,e,Forward charges
2,2001806229,0.50,b,Forward charges
3,2001806232,1.50,b,Forward charges
4,2001806233,0.50,b,Forward charges
...,...,...,...,...
119,2001821995,1.25,d,Forward charges
120,2001822466,1.50,b,Forward charges
121,2001823564,1.25,d,Forward and RTO charges
122,2001825261,2.50,d,Forward and RTO charges


In [428]:
for_cost['Payment Mode']=(pd.DataFrame(delivery_table['Payment Mode'].groupby(delivery_table.ExternOrderNo).first()).reset_index())['Payment Mode']

C:\Users\ykkou\AppData\Local\Temp\ipykernel_25028\1730359091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_cost['Payment Mode']=(pd.DataFrame(delivery_table['Payment Mode'].groupby(delivery_table.ExternOrderNo).first()).reset_index())['Payment Mode']


In [429]:
for_cost['Order Value']=(pd.DataFrame(delivery_table['Item Price(Per Qty.)'].groupby(delivery_table.ExternOrderNo).sum()).reset_index())['Item Price(Per Qty.)']

C:\Users\ykkou\AppData\Local\Temp\ipykernel_25028\3287348270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_cost['Order Value']=(pd.DataFrame(delivery_table['Item Price(Per Qty.)'].groupby(delivery_table.ExternOrderNo).sum()).reset_index())['Item Price(Per Qty.)']


In [430]:
for_cost

,ExternOrderNo,Weight Slab as per X (kg),Delivery Zone as per X,Type of Shipment,Payment Mode,Order Value
0,2001806210,0.50,b,Forward charges,COD,1324
1,2001806226,1.50,e,Forward charges,COD,1125
2,2001806229,0.50,b,Forward charges,COD,1082
3,2001806232,1.50,b,Forward charges,COD,2139
4,2001806233,0.50,b,Forward charges,Prepaid,1486
...,...,...,...,...,...,...
119,2001821995,1.25,d,Forward charges,Prepaid,484
120,2001822466,1.50,b,Forward charges,COD,9019
121,2001823564,1.25,d,Forward and RTO charges,Prepaid,4803
122,2001825261,2.50,d,Forward and RTO charges,Prepaid,9049


In [431]:
for_cost=for_cost.merge(courier_rates,left_on='Delivery Zone as per X',right_on='Zone',how='left')

In [432]:
for_cost

,ExternOrderNo,Weight Slab as per X (kg),Delivery Zone as per X,Type of Shipment,Payment Mode,Order Value,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,2001806210,0.50,b,Forward charges,COD,1324,b,0.50,33.0,28.3,20.5,28.3
1,2001806226,1.50,e,Forward charges,COD,1125,e,1.50,56.6,55.5,50.7,55.5
2,2001806229,0.50,b,Forward charges,COD,1082,b,0.50,33.0,28.3,20.5,28.3
3,2001806232,1.50,b,Forward charges,COD,2139,b,0.50,33.0,28.3,20.5,28.3
4,2001806233,0.50,b,Forward charges,Prepaid,1486,b,0.50,33.0,28.3,20.5,28.3
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1.25,d,Forward charges,Prepaid,484,d,1.25,45.4,44.8,41.3,44.8
120,2001822466,1.50,b,Forward charges,COD,9019,b,0.50,33.0,28.3,20.5,28.3
121,2001823564,1.25,d,Forward and RTO charges,Prepaid,4803,d,1.25,45.4,44.8,41.3,44.8
122,2001825261,2.50,d,Forward and RTO charges,Prepaid,9049,d,1.25,45.4,44.8,41.3,44.8


In [433]:
ship_type = for_cost['Type of Shipment']
forward_fixed = for_cost['Forward Fixed Charge']
forward_additional = for_cost['Forward Additional Weight Slab Charge']
rto_fixed = for_cost['RTO Fixed Charge']
rto_additional = for_cost['RTO Additional Weight Slab Charge']

In [434]:
conditions = [ship_type == 'Forward charges',ship_type =='Forward and RTO charges']
fixed_choices = [forward_fixed,forward_fixed+rto_fixed]
addi_choices = [forward_additional,forward_additional+rto_additional]

In [435]:
for_cost['Total fixed cost']=pd.DataFrame(np.select(conditions,fixed_choices,0))

In [436]:
for_cost['Total additional cost']=pd.DataFrame(np.select(conditions,addi_choices,0))

In [437]:
for_cost.columns

Index(['ExternOrderNo', 'Weight Slab as per X (kg)', 'Delivery Zone as per X',
       'Type of Shipment', 'Payment Mode', 'Order Value', 'Zone',
       'Weight Slabs', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge', 'Total fixed cost',
       'Total additional cost'],
      dtype='object')

In [438]:
for_cost.drop(['Forward Fixed Charge', 'Forward Additional Weight Slab Charge',
       'RTO Fixed Charge', 'RTO Additional Weight Slab Charge','Type of Shipment','Delivery Zone as per X'],axis=1,inplace=True)

In [439]:
for_cost

,ExternOrderNo,Weight Slab as per X (kg),Payment Mode,Order Value,Zone,Weight Slabs,Total fixed cost,Total additional cost
0,2001806210,0.50,COD,1324,b,0.50,33.0,28.3
1,2001806226,1.50,COD,1125,e,1.50,56.6,55.5
2,2001806229,0.50,COD,1082,b,0.50,33.0,28.3
3,2001806232,1.50,COD,2139,b,0.50,33.0,28.3
4,2001806233,0.50,Prepaid,1486,b,0.50,33.0,28.3
...,...,...,...,...,...,...,...,...
119,2001821995,1.25,Prepaid,484,d,1.25,45.4,44.8
120,2001822466,1.50,COD,9019,b,0.50,33.0,28.3
121,2001823564,1.25,Prepaid,4803,d,1.25,86.7,89.6
122,2001825261,2.50,Prepaid,9049,d,1.25,86.7,89.6


In [440]:
for_cost['w/o payment mode'] = (((for_cost['Weight Slab as per X (kg)']/for_cost['Weight Slabs'])-1)*for_cost['Total additional cost'])+for_cost['Total fixed cost']

In [441]:
for_cost.columns

Index(['ExternOrderNo', 'Weight Slab as per X (kg)', 'Payment Mode',
       'Order Value', 'Zone', 'Weight Slabs', 'Total fixed cost',
       'Total additional cost', 'w/o payment mode'],
      dtype='object')

In [442]:
for_cost.drop(['Weight Slab as per X (kg)', 'Zone',
       'Weight Slabs', 'Total fixed cost', 'Total additional cost'],axis=1,inplace=True)

In [443]:
for_cost

,ExternOrderNo,Payment Mode,Order Value,w/o payment mode
0,2001806210,COD,1324,33.0
1,2001806226,COD,1125,56.6
2,2001806229,COD,1082,33.0
3,2001806232,COD,2139,89.6
4,2001806233,Prepaid,1486,33.0
...,...,...,...,...
119,2001821995,Prepaid,484,45.4
120,2001822466,COD,9019,89.6
121,2001823564,Prepaid,4803,86.7
122,2001825261,Prepaid,9049,176.3


In [444]:
payment_mode = for_cost['Payment Mode']
order_valu = for_cost['Order Value']
pre_charge = for_cost['w/o payment mode']

In [445]:
conditions = [
    payment_mode == 'Prepaid',
    order_valu <= 300,
    order_valu > 300
]

choices = [
    pre_charge,pre_charge+15,pre_charge+(order_valu*0.05)
]


In [446]:
for_cost['Expected Charge as per X (Rs.)']= np.select(conditions,choices,0)

In [447]:
for_cost

,ExternOrderNo,Payment Mode,Order Value,w/o payment mode,Expected Charge as per X (Rs.)
0,2001806210,COD,1324,33.0,99.20
1,2001806226,COD,1125,56.6,112.85
2,2001806229,COD,1082,33.0,87.10
3,2001806232,COD,2139,89.6,196.55
4,2001806233,Prepaid,1486,33.0,33.00
...,...,...,...,...,...
119,2001821995,Prepaid,484,45.4,45.40
120,2001822466,COD,9019,89.6,540.55
121,2001823564,Prepaid,4803,86.7,86.70
122,2001825261,Prepaid,9049,176.3,176.30


In [448]:
table1['Expected Charge as per X (Rs.)'] = for_cost['Expected Charge as per X (Rs.)']

In [449]:
table1.drop('Type of Shipment',axis=1,inplace=True)

In [450]:
table1.columns

Index(['ExternOrderNo', 'AWB Code', 'Total Weight as per X(kg)',
       'Weight Slab as per X (kg)', 'Charged Weight', 'Delivery Zone as per X',
       'Zone', 'Billing Amount (Rs.)',
       'Weight Slab charged by Courier Company (Kg)',
       'Expected Charge as per X (Rs.)'],
      dtype='object')

In [451]:
table1 = table1.loc[:,['ExternOrderNo', 'AWB Code', 'Total Weight as per X(kg)',
       'Weight Slab as per X (kg)', 'Charged Weight','Weight Slab charged by Courier Company (Kg)','Delivery Zone as per X','Zone',
              'Expected Charge as per X (Rs.)','Billing Amount (Rs.)']]

In [452]:
table1

,ExternOrderNo,AWB Code,Total Weight as per X(kg),Weight Slab as per X (kg),Charged Weight,Weight Slab charged by Courier Company (Kg),Delivery Zone as per X,Zone,Expected Charge as per X (Rs.),Billing Amount (Rs.)
0,2001806210,1091117221940,0.220,0.50,2.92,3.00,b,b,99.20,174.5
1,2001806226,1091117222065,0.480,1.50,0.68,1.25,e,d,112.85,90.2
2,2001806229,1091117222080,0.500,0.50,0.71,1.25,b,d,87.10,90.2
3,2001806232,1091117222124,1.302,1.50,1.30,2.50,b,d,196.55,140.0
4,2001806233,1091117222135,0.245,0.50,0.78,1.00,b,b,33.00,61.3
...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1.25,0.50,1.25,d,d,45.40,45.4
120,2001822466,1091121305541,1.376,1.50,1.10,1.25,b,d,540.55,135.0
121,2001823564,1091121666133,0.672,1.25,0.70,1.25,d,d,86.70,172.8
122,2001825261,1091121981575,1.557,2.50,1.60,2.50,d,d,176.30,345.0


In [453]:
table1['Difference Between Expected Charges and Billed Charges (Rs.)'] = table1['Billing Amount (Rs.)']-table1['Expected Charge as per X (Rs.)']

In [454]:
table1.columns

Index(['ExternOrderNo', 'AWB Code', 'Total Weight as per X(kg)',
       'Weight Slab as per X (kg)', 'Charged Weight',
       'Weight Slab charged by Courier Company (Kg)', 'Delivery Zone as per X',
       'Zone', 'Expected Charge as per X (Rs.)', 'Billing Amount (Rs.)',
       'Difference Between Expected Charges and Billed Charges (Rs.)'],
      dtype='object')

In [455]:
table1.rename(columns = {'ExternOrderNo': 'Order ID',
                        'AWB Code':'AWB Number',
                        'Charged Weight':'Total Weight as per Courier Company (kg)',
                        'Zone':'Deliver Zone Charged by Courier Company',
                        'Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)'}, inplace = True)

In [456]:
table1

,Order ID,AWB Number,Total Weight as per X(kg),Weight Slab as per X (kg),Total Weight as per Courier Company (kg),Weight Slab charged by Courier Company (Kg),Delivery Zone as per X,Deliver Zone Charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.50,2.92,3.00,b,b,99.20,174.5,75.30
1,2001806226,1091117222065,0.480,1.50,0.68,1.25,e,d,112.85,90.2,-22.65
2,2001806229,1091117222080,0.500,0.50,0.71,1.25,b,d,87.10,90.2,3.10
3,2001806232,1091117222124,1.302,1.50,1.30,2.50,b,d,196.55,140.0,-56.55
4,2001806233,1091117222135,0.245,0.50,0.78,1.00,b,b,33.00,61.3,28.30
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1.25,0.50,1.25,d,d,45.40,45.4,0.00
120,2001822466,1091121305541,1.376,1.50,1.10,1.25,b,d,540.55,135.0,-405.55
121,2001823564,1091121666133,0.672,1.25,0.70,1.25,d,d,86.70,172.8,86.10
122,2001825261,1091121981575,1.557,2.50,1.60,2.50,d,d,176.30,345.0,168.70


In [457]:
table1.to_excel('Output Data 1.xlsx',index=False)

In [458]:
table2_count = [0,0,0]
table2_amount = [0,0,0]

In [459]:
for i in table1['Difference Between Expected Charges and Billed Charges (Rs.)']:
    if i > 0 :
        table2_count[1] += 1
    elif i == 0 :
        table2_count[0] += 1
    else:
        table2_count[2] += 1

In [460]:
table2_count

[8, 69, 47]

In [461]:
for i in range(len(table1['Difference Between Expected Charges and Billed Charges (Rs.)'])):
    if table1['Difference Between Expected Charges and Billed Charges (Rs.)'][i] > 0:
        table2_amount[1] += table1['Difference Between Expected Charges and Billed Charges (Rs.)'][i]
    elif table1['Difference Between Expected Charges and Billed Charges (Rs.)'][i] == 0:
         table2_amount[0] += table1['Charges Billed by Courier Company (Rs.)'][i]
    else :
        table2_amount[2] += table1['Difference Between Expected Charges and Billed Charges (Rs.)'][i]

In [462]:
table2_amount

[486.8999999999999, 3993.9000000000005, -3698.450000000001]

In [463]:
table2 = pd.DataFrame(list(zip(table2_count,table2_amount)),index=['Correctly Charged','Over Charged','Under Charged'],columns=['Count','Amount (Rs.)'])

In [464]:
table2

,Count,Amount (Rs.)
Correctly Charged,8,486.90
Over Charged,69,3993.90
Under Charged,47,-3698.45


In [465]:
table2.to_excel('Output Data 2.xlsx')